In [2]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy 
path = '/Users/dit/Documents/code/Learning/end-to-end-DE-project/postgres/yellow_tripdata_2021-01.parquet'
lu_path = '/Users/dit/Documents/code/Learning/end-to-end-DE-project/postgres/taxi+_zone_lookup.csv'

# View data

In [3]:
df = pd.read_parquet(path)
lu_df = pd.read_csv(lu_path)
print(df.shape)
print(lu_df.shape)
df.head()
lu_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dit/Documents/code/Learning/end-to-end-DE-project/postgres/yellow_tripdata_2021-01.parquet'

In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN


In [1]:
df.info()

NameError: name 'df' is not defined

# DB cred

In [3]:
user = 'postgres'
port = '5432'
host = 'localhost'
password = "***"
db = 'test_db'


# python ingest_data.py \
#     --user postgres \
#     --port 5432 \
#     --host localhost \
#     --password  ***\
#     --db test_db \
#     --table_name yellow_taxi_data_v1 \
#     --url /Users/dit/Documents/Learning/postgres/yellow_tripdata_2021-01.parquet
#     --url https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.parquet

In [4]:
engine = create_engine(f'postgresql+pg8000://{user}:{password}@{host}:{port}/{db}')
print(pd.io.sql.get_schema(df, name='yellow_taxi_data', con=engine))
print(pd.io.sql.get_schema(lu_df, name='lookup_table', con=engine))


CREATE TABLE yellow_taxi_data (
	"VendorID" BIGINT, 
	tpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	tpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	passenger_count FLOAT(53), 
	trip_distance FLOAT(53), 
	"RatecodeID" FLOAT(53), 
	store_and_fwd_flag TEXT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	payment_type BIGINT, 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	congestion_surcharge FLOAT(53), 
	airport_fee FLOAT(53)
)



CREATE TABLE lookup_table (
	"LocationID" BIGINT, 
	"Borough" TEXT, 
	"Zone" TEXT, 
	service_zone TEXT
)




# create table

In [5]:
df.head(2).to_sql(name='yellow_taxi_data', con=engine, if_exists='append', index=False)

2

In [9]:
df.head(5).to_sql(name='yellow_taxi_data_v2', con=engine, if_exists='replace', index=False)
# lu_df.to_sql(name='lookup_table', con=engine, if_exists='replace', index=False)


5

In [ ]:
def load_to_postgres(path):
    df = pd.read_parquet(path)
    engine = create_engine(f'postgresql+pg8000://{user}:{password}@{host}:{port}/{db}')

    while count < total:
        start = time()
        next_ = count + 50000
        # print(count, next_)
        write_df = df.iloc[count: next_]
        write_df.to_sql(name='yellow_taxi_data', con=engine, if_exists='append', index=False)
        count = next_
        print(f'inserted chunk... took: {round((time()-start), 2)} second')

# Load data into data in batches

In [10]:
from time import time
total = len(df)
count = 0


while count < total:
    start = time()
    next_ = count + 50000
    # print(count, next_)
    write_df = df.iloc[count: next_]
    write_df.to_sql(name='yellow_taxi_data', con=engine, if_exists='append', index=False)
    count = next_
    print(f'inserted chunk... took: {round((time()-start), 2)} second')


inserted chunk... took: 33.45 second
inserted chunk... took: 12.7 second
inserted chunk... took: 15.08 second
inserted chunk... took: 12.98 second
inserted chunk... took: 12.49 second
inserted chunk... took: 13.38 second
inserted chunk... took: 13.77 second
inserted chunk... took: 13.02 second
inserted chunk... took: 14.31 second
inserted chunk... took: 12.79 second
inserted chunk... took: 12.65 second
inserted chunk... took: 13.46 second
inserted chunk... took: 13.15 second
inserted chunk... took: 12.87 second
inserted chunk... took: 13.97 second
inserted chunk... took: 13.81 second
inserted chunk... took: 13.36 second
inserted chunk... took: 12.73 second
inserted chunk... took: 12.37 second
inserted chunk... took: 13.46 second
inserted chunk... took: 12.94 second
inserted chunk... took: 12.42 second
inserted chunk... took: 12.59 second
inserted chunk... took: 13.42 second
inserted chunk... took: 13.89 second
inserted chunk... took: 12.47 second
inserted chunk... took: 12.66 second
in

In [ ]:
total = len(df)
total

1369769

In [34]:
test[1].head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
10,2,2021-01-01 00:15:52,2021-01-01 00:38:07,3.0,6.11,1.0,N,164,255,1,20.5,0.5,0.5,0.00,0.00,0.3,24.30,2.5,NaN
11,2,2021-01-01 00:46:36,2021-01-01 00:53:45,2.0,1.21,1.0,N,255,80,1,7.0,0.5,0.5,2.49,0.00,0.3,10.79,0.0,NaN
12,1,2021-01-01 00:10:46,2021-01-01 00:32:58,2.0,7.40,1.0,N,138,166,2,24.5,2.5,0.5,0.00,6.12,0.3,33.92,0.0,NaN
13,2,2021-01-01 00:31:06,2021-01-01 00:38:52,5.0,1.70,1.0,N,142,50,1,8.0,0.5,0.5,2.36,0.00,0.3,14.16,2.5,NaN
14,2,2021-01-01 00:42:11,2021-01-01 00:44:24,5.0,0.81,1.0,N,50,142,2,4.5,0.5,0.5,0.00,0.00,0.3,8.30,2.5,NaN


In [36]:
1369769//50000

27

In [1]:
100000-190

99810